In [1]:
import os
from torch.utils.data import Dataset
from torchvision.io import read_image
import vcd.core as core
import pandas as pd

In [ ]:
def load_label_frames_list(label_path='./2_station_berliner_tor_2.1/2_station_berliner_tor_2.1_labels.json'):
    # Create empty vcd object
    myVCD = core.VCD()
    # Load a VCD file
    myVCD.load_from_file(label_path)

In [ ]:
lidar_path = './2_station_berliner_tor_2.1/lidar/'
frames_list = myVCD.data['openlabel']['frames']

In [ ]:
def generate_train_df (lidar_path, frames_list):
    anno_list = []    
    lidar_files = sorted(os.listdir(lidar_path))
    for lidar_file, frame in zip(lidar_files, frames_list):
        for objects in frames_list[frame]['objects']:
            object_data = frames_list[frame]['objects'][objects]['object_data']
            if 'cuboid' in object_data:
                cuboid = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['val']
                classes = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['name']
                anno = {}
                if lidar_file.startswith(str(frame)):
                    anno['filename'] = lidar_file
                anno['class'] = classes
                anno['cen_bb_x'] = cuboid[0]
                anno['cen_bb_y'] = cuboid[1]
                anno['cen_bb_z'] = cuboid[2]
                anno['vec_x'] = cuboid[3]
                anno['vec_y'] = cuboid[4]
                anno['vec_z'] = cuboid[5]
                anno['rota'] = cuboid[6]
                anno['bb_x'] = cuboid[7]
                anno['bb_y'] = cuboid[8]
                anno['bb_z'] = cuboid[9]
                anno_list.append(anno)
    return pd.DataFrame(anno_list)
        
generate_train_df(lidar_path, frames_list)

In [13]:
class LidarDataset(Dataset):
    def __init__(self, dir = "lidar", path = "/vol/coro/automatic_train_operation/OSDaR23/", transform=None, target_transform=None):
        # Labels
        # self.img_labels = pd.read_csv(annotations_file)

        self.path = path
        self.dir = dir
        self.transform = transform
        self.target_transform = target_transform
        self.labels = self.data_search()
        # df init
        for i in self.labels:
            self.df = generate_train_df(i[0], i[1])
            #df append
            # return df

    def data_search(self):
        path_list = []
        for station in os.listdir(self.path):
            path_final = os.path.join(self.path, station, self.dir)
            station_w_ending = station + ".json"
            label_frames = os.path.join(self.path, station, station_w_ending)
            path_list.append([path_final, label_frames])
        return path_list
    
    def generate_train_df (self, lidar_path, frames_path):
        # Create empty vcd object
        myVCD = core.VCD()
        # Load a VCD file
        myVCD.load_from_file(frames_path)
        frames_list = myVCD.data['openlabel']['frames']

        anno_list = []    
        lidar_files = sorted(os.listdir(lidar_path))
        for lidar_file, frame in zip(lidar_files, frames_list):
            for objects in frames_list[frame]['objects']:
                object_data = frames_list[frame]['objects'][objects]['object_data']
                if 'cuboid' in object_data:
                    cuboid = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['val']
                    classes = frames_list[frame]['objects'][objects]['object_data']['cuboid'][0]['name']
                    anno = {}
                    if lidar_file.startswith(str(frame)):
                        anno['filename'] = lidar_file
                    anno['class'] = classes
                    anno['cen_bb_x'] = cuboid[0]
                    anno['cen_bb_y'] = cuboid[1]
                    anno['cen_bb_z'] = cuboid[2]
                    anno['vec_x'] = cuboid[3]
                    anno['vec_y'] = cuboid[4]
                    anno['vec_z'] = cuboid[5]
                    anno['rota'] = cuboid[6]
                    anno['bb_x'] = cuboid[7]
                    anno['bb_y'] = cuboid[8]
                    anno['bb_z'] = cuboid[9]
                    anno_list.append(anno)
        return pd.DataFrame(anno_list)
    
    # label_path='./2_station_berliner_tor_2.1/2_station_berliner_tor_2.1_labels.json'

    

            # myVCD = core.VCD()
            # # Load a VCD file
            # myVCD.load_from_file(label_path)

        # # Image
        # path_list = []
        # for station in os.listdir(self.path):
        #     path_final = os.path.join(self.path, station, self.img_dir)

        #     for filename in os.listdir(path_final):
        #         path_filename = os.path.join(path_final, filename)

        #         # # checking if it is a file
        #         # if os.path.isfile(path_filename):
        #         #     print(path_filename)
        #         path_list.append(path_filename)


        # return path_list

    def __len__(self):
        return len(self.img_labels)

    def __getitem__(self, idx):
        img_path = os.path.join(self.img_dir, self.img_labels.iloc[idx, 0])
        image = read_image(img_path)
        label = self.img_labels.iloc[idx, 1]
        if self.transform:
            image = self.transform(image)
        if self.target_transform:
            label = self.target_transform(label)
        return image, label

In [14]:
a = LidarDataset()
a.data_search()

[['/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.1/lidar',
  '/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.1/09_station_ruebenkamp_9.1.json'],
 ['/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.2/lidar',
  '/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.2/09_station_ruebenkamp_9.2.json'],
 ['/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.3/lidar',
  '/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.3/09_station_ruebenkamp_9.3.json'],
 ['/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.4/lidar',
  '/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.4/09_station_ruebenkamp_9.4.json'],
 ['/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.5/lidar',
  '/vol/coro/automatic_train_operation/OSDaR23/09_station_ruebenkamp_9.5/09_station_ruebenkamp_9.5.json'],
 ['/vol/coro/automatic_train_operation/OSDaR23/09_station_ru